In [2]:
from simulator import blogm, bSqc, Neg, Sa
from mpo import tel_mpo
import torch
from math import prod
from functools import reduce
import pandas
from utils import dtype, device, pauli, basis, torch_data

In [3]:
seed = 0
train, test = False, True
file = f'seed{seed}'
train_ratio = 8/9
batch = 500

mdl = tel_mpo(34, bond=10)
total=0 # find size of the model
for p in mdl.parameters():
    total+=prod(p.shape)
total#, True_fid(mdl, psi)

6800

In [ ]:
all_N_values = []
all_S_values = []
all_I_values = []
for N in range(4, 36, 2):
    torch.manual_seed(seed)
    prepseq, shadow_state, rhoS = torch_data(f'../data/data_{N}na.pickle', shuffle=True)
    train_size = int(prepseq.shape[0]*train_ratio)
    test_size = prepseq.shape[0]-train_size
    
    prepseq_train, prepseq_test = prepseq[:train_size], prepseq[train_size:]
    shadow_state_train, shadow_state_test = shadow_state[:train_size], shadow_state[train_size:]
    rhoS_train, rhoS_test = rhoS[:train_size], rhoS[train_size:]
    
    # split in batches
    prepseq_train = prepseq_train.view(-1, batch, N-2)
    shadow_state_train = shadow_state_train.view(-1, batch, 4)
    rhoS_train = rhoS_train.view(-1, batch, 4, 4)

    prepseq_test = prepseq_test.view(-1, batch, N-2)
    shadow_state_test = shadow_state_test.view(-1, batch, 4)
    rhoS_test = rhoS_test.view(-1, batch, 4, 4)
    
    # load new model
    # if N == 4:
    #     mdl = tel_mpo(34, bond=10)
    # else:
    #     mdl.load_state_dict(torch.load(f'{file}/models/mpo_N={N-2}_na.pt'))
    
    # load old model
    mdl.load_state_dict(torch.load(f'{file}/models/mpo_N={N}_na.pt'))
    
    mps_N_values = []
    mps_S_values = []
    mps_I_values = []
    with torch.no_grad():
        print('='*50+'   Test   '+'='*50)
        mdl.eval()
        for i in range(prepseq_test.shape[0]):
            rhoC = mdl(prepseq_test[i])
            mps_N_values.append(Neg(rhoS_test[i], rhoC))
            mps_S_values.append(bSqc(rhoS_test[i], rhoC))
            mps_I_values.append(Sa(rhoS_test[i], rhoC)-bSqc(rhoS_test[i], rhoC))
    all_N_values.append(torch.cat(mps_N_values))
    all_S_values.append(torch.cat(mps_S_values))
    all_I_values.append(torch.cat(mps_I_values))
    print(f'N={N} done')
torch.save(torch.cat(all_N_values).view(16,-1), f'../../1d-result-figure/mps_N_values.pt')
torch.save(torch.cat(all_S_values).view(16,-1), f'../../1d-result-figure/mps_S_values.pt')
torch.save(torch.cat(all_I_values).view(16,-1), f'../../1d-result-figure/mps_I_values.pt')
    

==================================================   Test   ==================================================
N=4 done
==================================================   Test   ==================================================
N=6 done
==================================================   Test   ==================================================
N=8 done
==================================================   Test   ==================================================
N=10 done
==================================================   Test   ==================================================
N=12 done
==================================================   Test   ==================================================
N=14 done
==================================================   Test   ==================================================
N=16 done
==================================================   Test   ==================================================
N=18 done
===================================